In [1]:
import requests
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from datetime import datetime
import os

load_dotenv()
cricbuzz_api_key = os.getenv("CRICBUZZ_API_TOKEN")

In [2]:
@tool
def cricbuzz_data():
    #mandatory to give function description
    '''
    This function fetches data related to international cricket series
    '''
    
    url = "https://cricbuzz-cricket.p.rapidapi.com/series/v1/international"
    headers = {
    	"x-rapidapi-key": cricbuzz_api_key,
    	"x-rapidapi-host": "cricbuzz-cricket.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers)
    result = response.json()
    
    cricket_data = []
    data = result

    # Access each month and its series based on json data
    for month_data in data['seriesMapProto']:
        cricket_data.append(f"Month: {month_data['date']}")
        # Access each series in that month
        for series in month_data['series']:
            series = f"Series name: {series['name']}"
            cricket_data.append(series)
    return cricket_data

In [4]:
llm_model = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    temperature=0
)

In [5]:
llm_tools = llm_model.bind_tools([cricbuzz_data])

In [6]:
message = [HumanMessage('Can you provide me a list of internatinal cricket series from year 2026 onwards?')]

In [7]:
# response from llm
# this will contain that a tool is to be used as the question is related to the tool we have created
ai_message = llm_tools.invoke(message)
print(ai_message)

content='' additional_kwargs={'function_call': {'name': 'cricbuzz_data', 'arguments': '{}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--9759e5b5-16c6-4ec9-9f1f-b45d3c9d3990-0' tool_calls=[{'name': 'cricbuzz_data', 'args': {}, 'id': '715d7126-0b0e-4289-aa71-820088732567', 'type': 'tool_call'}] usage_metadata={'input_tokens': 54, 'output_tokens': 12, 'total_tokens': 331, 'input_token_details': {'cache_read': 0}}


In [8]:
# storing the history so the llm knows what is the context
message.append(ai_message)
print(message)

[HumanMessage(content='Can you provide me a list of internatinal cricket series from year 2026 onwards?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'cricbuzz_data', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--9759e5b5-16c6-4ec9-9f1f-b45d3c9d3990-0', tool_calls=[{'name': 'cricbuzz_data', 'args': {}, 'id': '715d7126-0b0e-4289-aa71-820088732567', 'type': 'tool_call'}], usage_metadata={'input_tokens': 54, 'output_tokens': 12, 'total_tokens': 331, 'input_token_details': {'cache_read': 0}})]


In [9]:
# if multiple tool calls happens then it will show list of all tools
ai_message.tool_calls

[{'name': 'cricbuzz_data',
  'args': {},
  'id': '715d7126-0b0e-4289-aa71-820088732567',
  'type': 'tool_call'}]

In [10]:
for tool_calling in ai_message.tool_calls:
    if tool_calling['name'] == 'cricbuzz_data':
        #execution of the tool if the tool name is called
        #can be used if else in case of multiple tools
        #passes args to the tool; in this case no use of argument so nothing is passed
        tool_msg = cricbuzz_data.invoke(tool_calling)
        message.append(tool_msg)

In [11]:
message

[HumanMessage(content='Can you provide me a list of internatinal cricket series from year 2026 onwards?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'cricbuzz_data', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--9759e5b5-16c6-4ec9-9f1f-b45d3c9d3990-0', tool_calls=[{'name': 'cricbuzz_data', 'args': {}, 'id': '715d7126-0b0e-4289-aa71-820088732567', 'type': 'tool_call'}], usage_metadata={'input_tokens': 54, 'output_tokens': 12, 'total_tokens': 331, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content=['Month: FEBRUARY 2024', 'Series name: ICC Cricket World Cup League Two 2023-27', 'Month: AUGUST 2025', 'Series name: South Africa tour of Australia, 2025', 'Series name: India tour of Bangladesh, 2025 (Called off)', 'Series name: Czech Republic tour of Romania 2025', 'Series name: Belgium tour of Aus

In [12]:
# push entire history to llm to extract only relevant context
final_result = llm_tools.invoke(message)

In [13]:
print(final_result.content)

Here is a list of international cricket series from 2026 onwards:

*   New Zealand tour of India, 2026
*   England tour of Sri Lanka 2026
*   West Indies tour of South Africa, 2026
*   South Africa tour of New Zealand, 2026
*   New Zealand tour of England, 2026
*   India tour of England, 2026
*   Pakistan tour of England 2026
*   Sri Lanka tour of England, 2026
